# Decorators:

These are higher-order functions - take other functions as input or/and output argument - that can modify the behaviour of other functions without changing the core functionality of the function of the function being decorated acts as wrapper. One can think of the decorator as a pizza topping and the decorated function as the dough. No matter the topping used the dough still remains a dough.  The decorator accepts other functions as its argument. decorators can be used for timing, caching and logging

Any suﬃciently generic functionality you can tack on to an existing class or function’s behavior makes a great use case for decoration. This includes the following:
* logging
* enforcing access control and authentication
* instrumentation and timing functions
* rate-limiting
* caching, and more


## Function decorators

In [1]:
def strong(func):
    def wrapper():
        return '<strong>' + func() + '</strong>'
    return wrapper
def emphasis(func):
    def wrapper():
        return '<em>' + func() + '</em>'
    return wrapper
@strong
@emphasis
def greet():
    return 'Hello!'

#This clearly shows in what order the decorators were applied: from bottom to top
greet()

'<strong><em>Hello!</em></strong>'

In [60]:
import time
import numpy as np

def timer(func):
    """Decorator for timing"""
    def logger():
        """Function that cal time"""
        start = time.time()
        re = func()
        end = time.time()
        print(f"Calling {func.__name__}: {end - start:.3f}")
        return re
    return logger
 
@timer
def cal_cum_sum():
    return np.cumsum(np.arange(10))
 
print(cal_cum_sum()) #what is happening under the hood is below

cal_cum_sum = timer(cal_cum_sum)
print(cal_cum_sum())

Calling cal_cum_sum: 0.000
[ 0  1  3  6 10 15 21 28 36 45]
Calling cal_cum_sum: 0.000
Calling logger: 0.000
[ 0  1  3  6 10 15 21 28 36 45]


## Class decorators

In [63]:
class timer_c(object):
    def __init__(self, func):
        self.func = func
    def __call__(self):
        start = time.time()
        re = self.func()
        end = time.time()
        print(f"Calling {self.func.__name__}: {end - start:.3f}")
        return re

@timer_c
def cal_cum_sum():
    return np.cumsum(np.arange(10))
 
print(cal_cum_sum())

Calling cal_cum_sum: 0.000
[ 0  1  3  6 10 15 21 28 36 45]


 The decorator above assumes that the decorated function requires no input as argument. Lets consider the case where an input is rquired nd how we can modify the decorator

In [59]:
@timer
def cal_cum_sum_m(m):
    return np.cumsum(np.arange(m))

cal_cum_sum_m(200)

TypeError: logger() takes 0 positional arguments but 1 was given

In [42]:
#here we use *args, **kwargs as place holders for an unspecified amount of position and keyword arguments

def timer_m(func):
    """Decorator for timing"""
    def logger(*args, **kwargs):
        """Function that cal time"""
        start = time.time()
        re = func(*args, **kwargs)
        end = time.time()
        print(f"Calling {func.__name__}: {end - start:.3f}")
        return re
    return logger

@timer_m
def cal_cum_sum_m(m):
    """cal the cumulative sum of array"""
    return np.cumsum(np.arange(m))

cal_cum_sum_m(10) 

Calling cal_cum_sum_m: 0.000


array([ 0,  1,  3,  6, 10, 15, 21, 28, 36, 45])

Decorator messes up the metadata/docstring of the decorated function as shown below and we see how this can be corrected for

In [43]:
print(cal_cum_sum_m.__doc__)

Function that cal time


In [44]:
from functools import wraps

def timer_m(func):
    """a decorator that prints how long a functio took to run.."""
    @wraps(func)
    def wrapper(*args, **kwargs):
        """Print 'hello' and then call the decorated function."""
        ts = time.time()
        re = func(*args, **kwargs)
        te = time.time() - ts
        print("{} took {}s".format(func.__name__, te))
        return re
    return wrapper

@timer_m
def cal_cum_sum_m(m):
    """cal the cumulative sum of array"""
    return np.cumsum(np.arange(m))

print(cal_cum_sum_m.__doc__)

cal the cumulative sum of array


### Some example decorators

In [45]:
def multiply(a,b):
    
    return a*b

def doubles(func):
    
    #defins a func we can modify
    def wrapper(a,b):
        
        return func(a*2, b*2)
    #returns the new func
    return wrapper

# taking a function as an argument
nw = doubles(multiply) # eqn 1

print(nw(1,4))

#another way to implemnent eqn 1 using
@doubles
def multiply(a,b):
    
    return a*b

print(multiply(1,4))

16
16


In [46]:
def print_before_and_after(func):
    def wrapper(*args):
        print('Before {}'.format(func.__name__))
        # Call the function being decorated with *args
        func(*args)
        print('After {}'.format(func.__name__))
        # Return the nested function
    return wrapper

@print_before_and_after
def multiply(a, b):
    print(a * b)

multiply(5, 10)

Before multiply
50
After multiply


In [47]:
def print_return_type(func):
    # Define wrapper(), the decorated function
    def wrapper(*args, **kwargs):
        # Call the function being decorated
        result = func(*args, **kwargs)
        print('{}() returned type {}'.format(
          func.__name__, type(result)
        ))
        return result
        # Return the decorated function
    return wrapper

@print_return_type
def foo(value):
    return value
  
print(foo(42))
print(foo([1, 2, 3]))
print(foo({'a': 42}))

foo() returned type <class 'int'>
42
foo() returned type <class 'list'>
[1, 2, 3]
foo() returned type <class 'dict'>
{'a': 42}


In [48]:
def counter(func):
  def wrapper(*args, **kwargs):
    wrapper.count += 1
    # Call the function being decorated and return the result
    return func(*args, **kwargs)
  wrapper.count = 0
  # Return the new decorated function
  return wrapper

# Decorate foo() with the counter() decorator
@counter
def foo():
  print('calling foo()')
  
foo()
foo()

print('foo() was called {} times.'.format(foo.count))

calling foo()
calling foo()
foo() was called 2 times.


In [71]:
def my_logger(orig_func):
    import logging
    logging.basicConfig(filename='{}.log'.format(orig_func.__name__), level=logging.INFO)

    @wraps(orig_func)
    def wrapper(*args, **kwargs):
        logging.info(
            'Ran with args: {}, and kwargs: {}'.format(args, kwargs))
        return orig_func(*args, **kwargs)

    return wrapper

@my_logger
def foo(*args,):
    return args
  
print(foo(42))


(42,)


* One downside of using decorators is that it “hides” some of the metadata attached to the original (undecorated) function. This makes debugging and working with the Python interpreter awkward and challenging. Thankfully there’s a quick ﬁx for this: the functools.wraps decorator included in Python’s standard library.4

In [8]:
def greet():
    """Return a friendly greeting."""
    return 'Hello!'
#print(greet.__name__, greet.__doc__)

def emphasis(func):
    def wrapper():
        return '<em>' + func() + '</em>'
    return wrapper

@emphasis
def greet():
    """Return a friendly greeting."""
    return 'Hello!'

print(greet.__name__, greet.__doc__)


wrapper None


In [7]:
import functools

def emphasis(func):
    @functools.wraps(func)
    def wrapper():
        return '<em>' + func() + '</em>'
    return wrapper

@emphasis
def greet():
    """Return a friendly greeting."""
    return 'Hello!'

print(greet.__name__, greet.__doc__)


greet Return a friendly greeting.


## Decorator with argument

In [49]:
def timer_m(unit:str):
    """Decorator for timing"""
    def logger(func):
        def inner_logger(*args, **kwargs):
            ts = time.time()
            re = func(*args, **kwargs)
            te = time.time() - ts
            para = 1e9 if unit =='ns' else 1 
            print(f"{func.__name__} took {te*para} {unit}")
            return re
        return inner_logger
    return logger

@timer_m('s')
def cal_cum_sum_m(m):
    """cal the cumulative sum of array"""
    return np.cumsum(np.arange(m))

cal_cum_sum_m(10) 

cal_cum_sum_m took 3.24249267578125e-05 s


array([ 0,  1,  3,  6, 10, 15, 21, 28, 36, 45])

In [50]:
def html(open_tag, close_tag):
  def decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
      msg = func(*args, **kwargs)
      return '{}{}{}'.format(open_tag, msg, close_tag)
    # Return the decorated function
    return wrapper
  # Return the decorator
  return decorator

# Make hello() return bolded text
@html('<b>', '</b>')
def hello(name):
  return 'Hello {}!'.format(name)
  
print(hello('Alice'))

<b>Hello Alice!</b>


In [51]:
def run_n_times(n):
    """define and return a decorator"""
    def decorator(func):
        def wrapper(*args, **kwargs):
            for _ in range(n):
                func(*args, **kwargs)
        return wrapper
    return decorator

@run_n_times(3)
def print_sum(a,b):
    print(a+b)
    
print_sum(3,4)   

7
7
7


In [52]:
# Use run_n_times() to create the run_five_times() decorator
run_five_times = run_n_times(5)

@run_five_times
def print_sum(a, b):
  print(a + b)
  
print_sum(4, 100)

104
104
104
104
104


In [1]:
def returns_dict(func):
  # Complete the returns_dict() decorator
  def wrapper(*args, **kwargs):
    result = func(*args, **kwargs)
    assert type(result) == dict
    return result
  return wrapper
  
@returns_dict
def foo(value):
    return value

try:
    print(foo([1,2,3]))
except AssertionError:
    print('foo() did not return a dict!')
#foo([1,2,3])  

foo() did not return a dict!
